In [1]:
import pandas as pd
from ast import literal_eval
from collections import Counter
import numpy as np

In [2]:
df = pd.read_csv("../results/cleaned_data.csv")

In [3]:
window = 10

In [4]:
word_list = df.noteTextList.tolist() 

In [5]:
# l1 = ["biden", "is", "a", "cat"]
# l2 = ["obama", "is", "a", "dog"]
# l3 = ["lucifer", "says", "hi", "dog"]
# lists = [l1,l2, l3]

In [6]:
def return_neighborhood_cts(word_list, i, window):
    # INPUT: a list of tokens, a pointer for the key word, window size
    # OUTPUT: a dictionary of counts for the neighborhood of the key word
    pre_pane_li = i-window
    pre_pane_ri = i
    post_pane_li = i+1
    post_pane_ri = i+1+window
    if pre_pane_li < window  and i < window:
        pre_pane_li = 0
    if i == len(word_list) - 1:
        pre_pane_li = len(word_list) - window - 1
    neighborhood = word_list[pre_pane_li:pre_pane_ri] + word_list[post_pane_li:post_pane_ri]
    inner_vecs = dict.fromkeys(set(neighborhood), 0)
    for word in neighborhood:
        inner_vecs[word] = inner_vecs[word] + 1
    return inner_vecs

master_class = {}
total_ct = 0
for l in word_list:
    word_list = literal_eval(l)
    outer_vec = {}
    for i in range(len(word_list)): # Looping thru all words
        tmp = return_neighborhood_cts(word_list, i, window) # returning neighborhood counts for a key word
        if (word_list[i] in outer_vec.keys()): # if a key word already has a dict entry, add to it
            for k, v in tmp.items():
                total_ct += v
                try:
                    outer_vec[word_list[i]][k] = outer_vec[word_list[i]][k] + v
                except:
                    outer_vec[word_list[i]][k] = v
        else: # if a key word does not have a dict entry, create it
            outer_vec[word_list[i]] = tmp
            total_ct += sum(tmp.values())
            
    # updating the master dict over many documents
    for k,v in outer_vec.items():
        try:
            master_class[k] = Counter(master_class[k]) + Counter(outer_vec[k])
        except:
            master_class[k] = outer_vec[k]




In [42]:
def npmi(w1, w2, vectors, vector_ct):
    # INPUT two words to compare, dict of vectors, total word count for denom
    # OUTPUT ppmi for those two words
    eps = 10**(-12)
    # numerator
    w1w2_dc = vectors[w1][w2] / vector_ct
    w1_dc = sum(vectors[w1].values()) / vector_ct
    w2_dc = sum(vectors[w2].values()) / vector_ct
    
    pmi_w1w2 = np.log((w1w2_dc) / ((w1_dc * w2_dc) + eps) + eps)
    npmi_w1w2 = pmi_w1w2 / (- np.log( (w1w2_dc) + eps))
    
    return npmi_w1w2

In [43]:
total_ct

3028007

In [49]:
npmi("twitter", "news", master_class, total_ct) 

0.024471896847835743

In [20]:
npmi("winning", "victory", master_class, total_ct)

<ipython-input-17-f92be535e946>:10: RuntimeWarning: divide by zero encountered in log
  top_pmi = np.log(top/((bottom_w1*bottom_w2)+eps))
<ipython-input-17-f92be535e946>:11: RuntimeWarning: divide by zero encountered in log
  denom = (-1*np.log(top))+eps
<ipython-input-17-f92be535e946>:12: RuntimeWarning: invalid value encountered in double_scalars
  npmi = top_pmi / denom


nan

In [37]:
w1 = "winning"
w2 = "victory"
vectors = master_class
vector_ct = total_ct

eps = 10**(-12)
# numerator
top = vectors[w1][w2] / (vector_ct + eps) 
bottom_w1 = sum(vectors[w1].values()) / vector_ct
bottom_w2 = sum(vectors[w2].values()) / vector_ct
# using natural log because that is what Bouma uses
top_pmi = np.log(top+eps/(bottom_w1*bottom_w2))
denom = (-1*np.log(top))+eps
npmi = top_pmi / denom

<ipython-input-37-e25250dabfe5>:13: RuntimeWarning: divide by zero encountered in log
  denom = (-1*np.log(top))+eps


In [38]:
top/(bottom_w1*bottom_w2)

0.0

In [ ]:
vectors[w2]

In [13]:
eps = 10**(-12)
eps

1e-12